# Getting Started with Gemini for Pair Programming

In [1]:
#!pip install python-dotenv
import os
from dotenv import load_dotenv, find_dotenv

def get_api_key():
    
    _ = load_dotenv(find_dotenv()) # read local .env file
    return os.getenv('GOOGLE_API_KEY')

To use Gemini API on your own machine, you would first install the library:
```Python
!pip install -q google.generativeai
```
The optional flag `-q` installs "quietly" without printing out details of the installation.


> Note: Get the Gemini API from this [website](https://aistudio.google.com/app/apikey) by only using your gmail account.

```Python
# Legacy PALM API code shown in the video
import google.generativeai as palm
palm.configure(api_key=get_api_key())
```

In [2]:
# From now own with Gemini API
import os
import google.generativeai as genai
from google.api_core import client_options as client_options_lib

genai.configure(
    api_key=get_api_key(),
    transport="rest",
    client_options=client_options_lib.ClientOptions(
        api_endpoint=os.getenv("GOOGLE_API_BASE"),
    )
)

### Explore the available models

In [4]:
# PaLM API legacy:
# palm.list_models()

# Now, Gemini API
for m in genai.list_models():
    print(f"name: {m.name}")
    print(f"description: {m.description}")
    print(f"generation methods:{m.supported_generation_methods}\n")

name: models/chat-bison-001
description: A legacy text-only model optimized for chat conversations
generation methods:['generateMessage', 'countMessageTokens']

name: models/text-bison-001
description: A legacy model that understands text and generates text as an output
generation methods:['generateText', 'countTextTokens', 'createTunedTextModel']

name: models/embedding-gecko-001
description: Obtain a distributed representation of a text.
generation methods:['embedText', 'countTextTokens']

name: models/gemini-1.0-pro-latest
description: The original Gemini 1.0 Pro model. This model will be discontinued on February 15th, 2025. Move to a newer Gemini version.
generation methods:['generateContent', 'countTokens']

name: models/gemini-1.0-pro
description: The best model for scaling across a wide range of tasks
generation methods:['generateContent', 'countTokens']

name: models/gemini-pro
description: The best model for scaling across a wide range of tasks
generation methods:['generateCon

#### Filter models by their supported generation methods
- `generateText` is currently recommended for coding-related prompts.
- `generateMessage` is optimized for multi-turn chats (dialogues) with an LLM.

- `generateContent`, best model for scaling across a wide range of tasks.

In [5]:
models = [m for m in genai.list_models() 
          if 'generateText' 
          in m.supported_generation_methods]
models

[Model(name='models/text-bison-001',
       base_model_id='',
       version='001',
       display_name='PaLM 2 (Legacy)',
       description='A legacy model that understands text and generates text as an output',
       input_token_limit=8196,
       output_token_limit=1024,
       supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
       temperature=0.7,
       top_p=0.95,
       top_k=40)]

In [6]:
# Model Bison in 2024 is a legacy model  
model_bison = models[0]
model_bison

Model(name='models/text-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 (Legacy)',
      description='A legacy model that understands text and generates text as an output',
      input_token_limit=8196,
      output_token_limit=1024,
      supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
      temperature=0.7,
      top_p=0.95,
      top_k=40)

#### helper function to generate text

- The `@retry` decorator helps you to retry the API call if it fails.
- We set the temperature to 0.0 so that the model returns the same output (completion) if given the same input (the prompt).

```Python
# Code legacy for PALM API
from google.api_core import retry
@retry.Retry()
def generate_text(prompt,
                  model=model_bison,
                  temperature=0.0):
    return palm.generate_text(prompt=prompt,
                              model=model,
                              temperature=temperature)
```

In [7]:
# Set the model to connect to the Gemini API
model_flash = genai.GenerativeModel(model_name='gemini-1.5-flash')

In [8]:
# Helper with Gemini API
def generate_text(prompt,
                  model=model_flash,
                  temperature=0.0):
    return model_flash.generate_content(prompt,
                                  generation_config={'temperature':temperature})

#### Ask the LLM how to write some code



In [9]:
prompt = "Show me how to iterate across a list in Python."

In [10]:
# Gemini API updates to generate the text
completion = generate_text(prompt)

In [11]:
# PaLM API
## print(completion.result)

# Gemini API
print(completion.text)

Python offers several ways to iterate across a list. Here are the most common methods, with explanations and examples:

**1. Using a `for` loop:** This is the most straightforward and commonly used method.

```python
my_list = ["apple", "banana", "cherry"]

# Iterate and print each item
for item in my_list:
    print(item)

# Iterate and print the index and item
for i, item in enumerate(my_list):
    print(f"Item at index {i}: {item}")
```

**Explanation:**

* The first loop directly iterates through each element in `my_list`, assigning it to the variable `item` in each iteration.
* The second loop uses `enumerate()`, which provides both the index (`i`) and the value (`item`) of each element in the list.  `f-strings` (formatted string literals) are used for cleaner output.


**2. Using a `while` loop:** This offers more control, but is generally less concise for simple list iteration.

```python
my_list = ["apple", "banana", "cherry"]
i = 0
while i < len(my_list):
    print(my_list[i])

- **Tip:** The words "show me" tends to encourage the ~~PaLM~~ Gemini LLM to give more details and explanations compared to if you were to ask "write code to ..."

In [12]:
prompt = "write code to iterate across a list in Python"

In [13]:
# PaLM API
## completion = generate_text(prompt)
## print(completion.result)

# Gemini API
completion = generate_text(prompt)
print(completion.text)

There are several ways to iterate across a list in Python, each with its own advantages:

**1. Using a `for` loop:** This is the most common and generally preferred method for iterating through a list.

```python
my_list = [10, 20, 30, 40, 50]

# Iterate and print each element
for item in my_list:
    print(item)

# Iterate and print the index and element
for i, item in enumerate(my_list):
    print(f"Index: {i}, Value: {item}")
```

**2. Using a `while` loop:** This offers more control, allowing you to iterate based on a condition rather than just the length of the list.  However, it's generally less readable and more prone to errors than a `for` loop for simple list iteration.

```python
my_list = [10, 20, 30, 40, 50]
i = 0
while i < len(my_list):
    print(my_list[i])
    i += 1
```

**3. List comprehension (for creating new lists):**  List comprehensions are concise ways to create new lists based on existing ones.  While not strictly iteration in the same sense as `for` and `while`

#### Try out the code
- Try copy-pasting some of the generated code and running it in the notebook.
- Remember to test out the LLM-generated code and debug it make sure it works as intended.

In [14]:
my_list = [10, 20, 30, 40, 50]
my_iterator = iter(my_list)

try:
    while True:
        item = next(my_iterator)
        print(item)
except StopIteration:
    pass




10
20
30
40
50


#### Try asking your own coding question

In [15]:
# Modify the prompt with your own question
prompt = "Show me how to calcutate average using python"

# PaLM API
## completion = generate_text(prompt)

# Gemini API
completion = generate_text(prompt)

In [19]:
print(completion.text)

There are several ways to calculate the average (mean) in Python, depending on your data structure. Here are a few examples:

**1. Using the `statistics` module (Recommended for simplicity and efficiency):**

This is the easiest and most efficient way, especially for larger datasets.  The `statistics` module provides functions specifically designed for statistical calculations.

```python
import statistics

data = [1, 2, 3, 4, 5]
average = statistics.mean(data)
print(f"The average is: {average}")  # Output: The average is: 3

data2 = [10, 20, 30, 40, 50, 100]
average2 = statistics.mean(data2)
print(f"The average is: {average2}") # Output: The average is: 40.0

#Handles empty lists gracefully
empty_data = []
try:
    average3 = statistics.mean(empty_data)
    print(f"The average is: {average3}")
except statistics.StatisticsError:
    print("Cannot calculate the average of an empty list.") #Output: Cannot calculate the average of an empty list.

```

**2. Using `sum()` and `len()` (For b

#### Note about the API key
We've provided an API key for this classroom.  If you would like your own API key for your own projects, you can get one at [developers.generativeai.google](https://developers.generativeai.google/)